# 네이버 지도 크롤링

검색어에 따른 장소에 대한 정보를 크롤링하는 코드입니다.   
장소의 이름, 전화번호, 주소, 평점, 리뷰정보에 대해 크롤링합니다.   
장소에 대한 간단한 정보들을 담은 data, 리뷰에 대한 정보들을 담은 data, 블로그 주소에 대한 data를 가져옵니다.   
(현재 코드는 '카페'정보를 가져오기 위해 작성되었습니다)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import datetime

# 특정 div scroll down function
def doScrollDown(whileSeconds):
    start = datetime.datetime.now()
    end = start + datetime.timedelta(seconds=whileSeconds)
    cont = driver.find_element_by_id('_pcmap_list_scroll_container')
    
    while True:
        driver.execute_script("arguments[0].scrollBy(0, 10000)", cont)
        time.sleep(1)
        if datetime.datetime.now() > end:
            break


# 네이버 지도는 접속 시 자동으로 아래 url 뒤에 현재 위치 정보가 입력됨
driver_path = './driver/chromedriver'
URL = 'https://map.naver.com/v5/'
search_keyword = '카페'

driver = webdriver.Chrome(executable_path=driver_path)
driver.get(URL)

time.sleep(5)

# keyword 검색
search = driver.find_element_by_css_selector('div.input_box>input.input_search')
search.send_keys(search_keyword)
search.send_keys(Keys.ENTER)

time.sleep(3)

# frame 전환
search_frame = driver.find_element_by_id('searchIframe')
driver.switch_to.frame(search_frame)
doScrollDown(5)


# 탐색은 페이지 별로 진행합니다.
search_simple_result = []
search_review_result = []
search_blog_url_result = []
cur_page = 1
while True:
    places = driver.find_elements_by_class_name('_3nCBm')
    for pl in places:
        # 검색할 장소가 있다면 상세페이지를 엽니다.
        try:
            pl.click()
        except:
            continue
        time.sleep(3)

        # 장소 상세정보 frame으로 전환
        driver.switch_to.default_content()
        entry_frame = driver.find_element_by_id('entryIframe')
        driver.switch_to.frame(entry_frame)


        # 장소의 기본정보
        # {이름:name , 전화번호:number , 주소:addr , 평점:score , 방문자수:cnt_visitor , 블로그수:cnt_blog}
        name = driver.find_element_by_class_name('_3XamX').text
        try:
            number = driver.find_element_by_class_name('_3ZA0S').text
        except:
            number = 'none'
        addr = driver.find_element_by_class_name('_2yqUQ').text
        try:
            score = driver.find_element_by_xpath('//*[@id="app-root"]/div/div/div[2]/div[1]/div/div/div[1]/div/span[1]/em').text
        except:
            score = 0
        try:
            cnt_visitor = driver.find_element_by_xpath('//*[@id="app-root"]/div/div/div[2]/div[1]/div/div/div[1]/div/span[2]/a/em').text
        except:
            cnt_visitor = 0
        try:
            cnt_blog = driver.find_element_by_xpath('//*[@id="app-root"]/div/div/div[2]/div[1]/div/div/div[1]/div/span[3]/a/em').text
        except:
            cnt_blog = 0


        # 방문자 리뷰정보 [review_info]
        if cnt_visitor != 0:
            driver.find_elements_by_class_name('_2RG_o')[-2].click()
            time.sleep(3)
            # 모든 리뷰정보를 로드합니다.
            while True:
                try:
                    driver.find_element_by_link_text('더보기').click()
                    time.sleep(1)
                except:
                    break
            while True:
                try:
                    driver.find_element_by_class_name('_2vy4Q').click()
                    time.sleep(1)
                except:
                    break
            # 모든 리뷰글 정보들을 가져옵니다.
            review_text = driver.find_elements_by_class_name('_3vfQ6')
            for per in review_text:
                search_review_result.append([name, per.text])


        # 블로그 리뷰정보 [blog_url]
        if cnt_blog != 0:
            driver.find_elements_by_class_name('_3Q5_9')[-1].click()
            time.sleep(3)
            # 모든 리뷰정보를 로드합니다.
            while True:
                try:
                    driver.find_element_by_link_text('더보기').click()
                    time.sleep(1)
                except:
                    break
            # 모든 리뷰 블로그 url들을 가져옵니다.
            review_url = driver.find_elements_by_class_name('awH2_')
            for per in review_url:
                search_blog_url_result.append([name, per.find_element_by_tag_name('a').get_attribute('href')])


        # 현재 장소에 대한 정보 저장
        search_simple_result.append([name, number, addr, score, cnt_visitor, cnt_blog])
        # 정보 저장 후 검색 프레임으로 이동
        driver.switch_to.default_content()
        search_frame = driver.find_element_by_id('searchIframe')
        driver.switch_to.frame(search_frame)
    
    
    # 다음 페이지가 존재한다면 이어서 탐색합니다.
    cur_page = cur_page + 1
    print('check', cur_page)
    page_div = driver.find_element_by_class_name('_2ky45')
    try:
        page_div.find_element_by_link_text(str(cur_page)).click()
    except:
        break
    time.sleep(3)
    
    
# 검색 결과 모두 조회
# result_container = driver.find_element_by_id('_pcmap_list_scroll_container')
# result_container
# 각 url내에서 데이터 크롤링 진행
# 혹은 검색 결과창에서 그대로 크롤링

# 크롤링 한 데이터 저장

# driver 종료
driver.quit()

정상적으로 크롤링 되었는지 가져온 데이터들을 확인합니다.

In [ ]:
print(len(search_simple_result))
print(search_simple_result)

In [ ]:
print(len(search_review_result))
print(search_review_result)

In [ ]:
print(len(search_blog_url_result))
print(search_blog_url_result)

DB에 저장하기 전 리뷰 데이터에서 이모지들을 제거합니다.

In [ ]:
for i in range(len(search_review_result)):
    search_review_result[i][1] = search_review_result[i][1].encode('cp949', 'ignore').decode('cp949')

# 데이터 저장

sqlite3를 이용해 크롤링한 데이터들을 저장합니다.   
- simple_info_table : 장소에 대한 간단한 정보들
- review_info_table :리뷰에 대한 정보들
- blog_url_table : 블로그 링크에 대한 정보들

In [ ]:
import pandas as pd
import sqlite3

simple_result = pd.DataFrame(search_simple_result)
simple_result.columns = ['이름', '전화번호', '주소', '평점', '방문자수', '블로그리뷰수']

review_result = pd.DataFrame(search_review_result)
review_result.columns = ['이름', '리뷰']

blog_url_result = pd.DataFrame(search_blog_url_result)
blog_url_result.columns = ['이름', 'URL']


conn = sqlite3.connect('./navermap_search_result.db')
simple_result.to_sql('simple_info_table', conn, chunksize=1000)
review_result.to_sql('review_info_table', conn, chunksize=1000)
blog_url_result.to_sql('blog_url_table', conn, chunksize=1000)
conn.commit()
conn.close()

저장이 잘 되었는지 쿼리문을 통해 확인합니다.

In [ ]:
conn = sqlite3.connect('navermap_search_result.db')
simple = pd.read_sql('select * from simple_info_table', conn)
review = pd.read_sql('select * from review_info_table', conn)
blog_url = pd.read_sql('select * from blog_url_table', conn)
conn.commit()
conn.close()

In [ ]:
simple

In [ ]:
review

In [ ]:
blog_url

# 블로그 리뷰 크롤링

앞서 크롤링한 블로그 링크 정보들을 이용해 블로그 리뷰 데이터들을 가져옵니다.

In [ ]:
import pandas as pd
import sqlite3

# DB에서 블로그 링크 정보를 가져옵니다.
conn = sqlite3.connect('./navermap_search_result.db')
blog_url = pd.read_sql('select * from blog_url_table', conn)
conn.commit()
conn.close()

blog_url

In [ ]:
print(len(blog_url))

for i in range(len(blog_url)):
    if 'blog' in blog_url.loc[i][2]:
        a = 1
    elif 'cafe' in blog_url.loc[i][2]:
        print(blog_url.loc[i][2])
    else:
        print(blog_url.loc[i][2])

blog가 아닌 url은 iframe 및 태그 정보가 달라 탐색에서 제외합니다.   
개수 또한 기존 1022개의 데이터 중 10개 정도 해당하기 때문에 제외하여도 결과에 미치는 영향은 미미할 것입니다.

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import datetime
import requests

driver_path = './driver/chromedriver'
driver = webdriver.Chrome(executable_path=driver_path)

blog_text = []
for i in range(len(blog_url)):
    name = blog_url.loc[i][1]
    url = blog_url.loc[i][2]

    page_type = 'none'
    if not 'blog' in url:
        continue
    
    driver.get(url)
    time.sleep(3)

    # frame 전환
    search_frame = driver.find_element_by_id('mainFrame')
    driver.switch_to.frame(search_frame)
    
    btxt = driver.find_element_by_id('post_1').text

    blog_text.append([name, btxt])

장소의 이름과 해당 장소에 대한 모든 blog의 text들을 가져왔습니다.   
이를 DB에 저장하기 전 불필요한 앞부분과 뒷부분들을 제거합니다.

In [ ]:
blog_text

먼저 text를 보면 이웃추가가 반복되는 것으로 보입니다.   
이를 기준으로 모든 text에서 동일한 패턴이 있는지 확인합니다.

In [ ]:
no_with = []
for per in blog_text:
    if not '이웃추가' in per:
        no_with.append(per)
len(no_with)

In [ ]:
cnt = 0
for per in no_with:
    if not '지도로 보기' in per:
        cnt = cnt + 1
cnt

탐색 결과 blog에서 가져온 text들은 '이웃추가'가 상단에 없는 text도 있어, 대신 '지도로 보기'로 탐색하니 모든 경우를 고려하게 됩니다.   
'이웃추가'와 '지도로 보기'를 이용해 blog 데이터에서 필요없는 앞 부분을 제거합니다.

In [ ]:
tt[0].find('지도로 보기')
# # 지도로 보기는 6글자이므로 위 결과에 +7이 시작 부분
# 95이면 102부터

tt[1].find('이웃추가')
# 이웃추가는 4글자이므로 +4가 시작 부분
# 55이면 59부터

불필요한 뒷부분은 주로 검색을 위한 태그로 시작되는 것으로 보여 이를 기준으로 제거합니다.

In [ ]:
import re

# 정규표현식으로 마지막 태그 이후 부분을 모두 제거
m = re.findall(r'[#]\w+', tt[10])
ff = tt[10].find(m[0])
tt[10][ff:ff+1000]

마무리로 text에서 불필요한 문자들(줄넘김 등)을 제거합니다.

전처리한 blog 리뷰 데이터들을 DB에 저장합니다.

In [ ]:
conn = sqlite3.connect('./navermap_search_result.db')
blog_text.to_sql('blog_info_table', conn, chunksize=1000)
conn.commit()
conn.close()

DB에 저장이 잘 되었는지 확인합니다.

In [ ]:
conn = sqlite3.connect('./navermap_search_result.db')
blog = pd.read_sql('select * from blog_info_table', conn)
conn.commit()
conn.close()